### DTS - Complete Streets

# 07 - Street Typology Overrides

**Author:** rmangan

---

**Purpose:**

After intial assingment of Complete Street Typologies, data was published to ArcGIS Online for review by DTS Staff. DTS Staff provided input on individual streets that were mis-assigned. Input was provided in the form of an ArcGIS Online point feature service, configured for editing by DTS staff with options to select a street type "override". Points were placed on the map on streets that needed adjustment.

**This script performs the following functions:**

1. Download AGOL comment layer via ArcGIS API for python

2. Spatial join comment point layer to modal composite layer.

3. Update complete street typology in moodal composite based on complete street type in comment point layer. If DTS staff entered verbiage into the "Note" field, the comment is skipped and must be manually addressed/edited in ArcGIS Pro.


**Global Assumptions and Notes:**
1. All typology comments with values in "Note" field are skipped, comments must be manually addressed in ArcGIS Pro.
2. item
3. item
4. item

**Non-Standard Python Modules utilized:**
1. arcpy 2.7 - used for common data geo-processing
2. arcgis 1.83 - used for connecting to AGOL org and related items
2. pandas 1.1 - (optional) used for QC purposess and data exploration

In [ ]:
# import modules
import arcpy
import os
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS

In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\typology_comments.gdb"
arcpy.env.overwriteOutput = True

# login to AGOL
my_gis = GIS()

print("ArcGIS Online Org account")

pw = input("Enter AGOL password...")

gis = GIS("http://dithnl.maps.arcgis.com", "ray.mangan_ch2m", pw)
print("Logged in as " + str(gis.properties.user.username))


In [ ]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

modal_gdb = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb"

comment_gdb = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\typology_comments.gdb"


# Input Datasets

#REST URL for comment layer:

comment_REST = "https://services6.arcgis.com/2cZSk3EXXiOHcbOl/arcgis/rest/services/Complete_Streets_Comments/FeatureServer/0"

modal_composite = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb\modal_composite_05_3"



## 01 - Section name
---

Assign Complete Street typolgies based on attrbutes in modal composite dataset, complete streets design manaual, and direct input from project stakeholders.

**This section performs the following functions:**

1. Calculate % of each line segment, by length, that intersects with Urban land use zones

2. Calculate % of each line segment, by length, that intersects with Agricultural land use zones.




**Assumptions and Notes:**
1. Land Use Zoning from:


3. Agricultural land use zones
4. item

In [ ]:
#copy cs layer to local gdb

comment_local = os.path.join(comment_gdb,"typology_comments_20210723")

print("Copying Comment layer to local gdb...")
#arcpy.CopyFeatures_management(comment_REST, comment_local)
print("done")

#spatial join cs comments to rcl
sj_output = os.path.join(modal_gdb, "sj_output")

print("Running spatial join...")
arcpy.analysis.SpatialJoin(modal_composite, comment_local, sj_output, join_operation = "JOIN_ONE_TO_ONE", match_option = "CLOSEST", search_radius = 10)
print("done")

#select records where comment note = null

#update cs type with comment type


In [ ]:
#featureset and updte testing against AGOL FS

#generate token
token = gis._con.token

print("Token: {}".format(str(token)))

modal_composite_fs = modal_composite_url_copy + "?token={}".format(token)

print(modal_composite_fs)

featureset = arcpy.FeatureSet()

fs = gis.content.get('6655329b1fb34a91b419b8d002588524')
fs.url

# lyr = arcpy.management.MakeFeatureLayer(modal_composite, "modal_comp_lyr")[0]


featureset.load(modal_composite_fs)

In [ ]:
type(modal_composite_fs)

In [ ]:
fields = ["SEGMENTID","FULLNAME"]

count = 0

with arcpy.da.SearchCursor(modal_composite_fs, fields) as cursor:
    for row in cursor:
        print(row[1])
        count += 1

        
print(count)
        

In [ ]:
median_df = pd.DataFrame.spatial.from_featureclass(medians)

median_fields = median_df.columns.values.tolist()

median_fields

In [ ]:
fields = ["SEGMENTID","FULLNAME","CLASS","ONEWAY","mean_row_2020","AB_LANE","BA_LANE","ln_exist_num","TPN_OID","NewTier","modal_width_ped_exist"]

filt = (sdf["modal_width_ped_exist"]>0)

#two way streets, null lane count, row > 100
filt2 = ((sdf["AB_LANE"].isnull()) &
         (sdf["BA_LANE"].isnull()) &
         (sdf["ONEWAY"] == 0) &
         (sdf["mean_row_2020"]>100))
         
#one way streets, null lane count, row > 75
filt3 = ((sdf["AB_LANE"].isnull()) &
         (sdf["BA_LANE"].isnull()) &
         (sdf["ONEWAY"].isin([1,2])) &
         (sdf["mean_row_2020"]>75))

filt4 = (sdf["CLASS"] == 'A41')

filt5 = ((~sdf["TPN_OID"].isnull()) &
         (sdf["NewTier"] == 4))

filt_CSDM = ((~sdf["CSDM_EXAMPLE"].isnull()))


pd.set_option('display.max_rows', 250)
sdf.loc[filt5,fields].sort_values(by="FULLNAME").shape[0]

In [ ]:
#pd.set_option('display.max_rows', 250)
sdf.loc[filt2,fields].sort_values(by="FULLNAME")

In [ ]:
#Create DF for CSDM analysis
cs_df = pd.DataFrame.spatial.from_featureclass(modal_composite_05a)

cs_df_fields = cs_df.columns.values.tolist()

cs_df_fields.sort()

for i in cs_df_fields:
    print(i)

In [ ]:
#Set CSDM analysis fields and filters
df_fields = ["SEGMENTID","FULLNAME","CS_example_num","AB_SPEED","BA_SPEED","AB_LANE","BA_LANE","TOT_FLOW_D", "mean_row_2020"]

filt = ((~cs_df["CS_example_num"].isnull()))

filt35_CS = ((cs_df["AB_SPEED"] == 35 ))

filt_row = ((cs_df["mean_row_2020"] < 7 ))

filt_examples = ((~cs_df["CS_example_num"].isnull()))

filt_blvd = (((cs_df["AB_SPEED"] > 35 ) |
             (cs_df["BA_SPEED"] > 35 )) &
             (cs_df["TOT_FLOW_D"] > 10000) &
             ((cs_df["AB_LANE"] + cs_df["BA_LANE"]) >= 4 )
             & ((~cs_df["CS_example_num"].isnull()))  
            )

filt_ave = (((cs_df["AB_SPEED"] < 35 ) |
             (cs_df["BA_SPEED"] < 35 )) &
             (cs_df["TOT_FLOW_D"] > 10000) &
             ((cs_df["AB_LANE"] + cs_df["BA_LANE"]) <= 4 )
             #& ((~cs_df["CS_example_num"].isnull()))  
            )

In [ ]:
#pd.set_option('display.max_rows', 250)

#cs_df.loc[filt35_CS,df_fields].sort_values(by="FULLNAME")

#cs_df.filter(df_fields)

#cs_df.loc[(cs_df["FULLNAME"] == "N KING ST"),df_fields]

cs_df.loc[filt_row, df_fields]

In [ ]:
#CSDM Street Typology Pre-Processing - REMOVE FROM FINAL CODE


#join CSDM exmaple values into dataset
#join fields from median classification
print("Joining CSDM Example field...")
join_target = modal_composite_05a
join_target_field = "SEGMENTID"
join_table = modal_composite_02
join_table_field = "SEGMENTID"
join_fields = ["CSDM_EXAMPLE"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")
    
print("Done")


#alter joined field to match naming conventiones
arcpy.management.AlterField(modal_composite_05a,"CSDM_EXAMPLE", new_field_name = "CS_example_num", new_field_alias = "Complete Streets - Example Number" )

#add CS example type field
arcpy.AddField_management(modal_composite_05a,field_name="CS_example_type",field_type="TEXT",field_alias = "Complete Streets - Example Type", field_length = 30)

#add CS type field
arcpy.AddField_management(modal_composite_05a,field_name="CS_type",field_type="TEXT",field_alias = "Complete Streets - Type", field_length = 30)




In [ ]:
#Compute Urban zoning line percentage metric

#select appropriate zoning categories, using the following ZONE_CLASS values

# Aloha - State Jurisdiction: Aloha Tower Project (Admin. by Aloha Tower Development Corp.)
# AMX-2
# AMX-3
# B-1 - B-1 Neighborhood Business District
# B-2 - B-2 Community Business District
# BMX-3 - BMX-3 Community Business Mixed Use District
# BMX-4 - BMX-4 Central Business Mixed Use District
# Kak - State Jurisdiction: Kakaako Community Development District (Admin. by HCDA)
# MU - Mixed Use Precinct (Kakaako Special Design District)
# PU - Public Use Precinct (Kakaako Special Design District)
# Pub - Public Precinct (Waikiki Special District)
# ResMix - Resort Mixed Use Precinct (Waikiki Special District)
# Resort - Resort District

print("create urban zoning layer...")
urban_zoning = arcpy.management.MakeFeatureLayer(zoning, "urban_zoning", where_clause='"ZONE_CLASS" IN(\'Aloha\', \'AMX-2\', \'AMX-3\', \'B-1\', \'B-2\', \'BMX-3\', \'BMX-4\', \'Kak\', \'MU\', \'PU\', \'Pub\', \'ResMix\', \'Resort\')' )

print("dissolve zoning...")
dissolve_output = os.path.join(scratch_gdb_path, "urban_zoning_dissolve")

urban_zoning_dissolve = arcpy.management.Dissolve(urban_zoning, dissolve_output)

print("intersect zoning w/ modal_comp...")
# intersect modal comp w/ selected zoning features
intersect_output = os.path.join(scratch_gdb_path, "modal_zoning_intersect")

modal_zoning_intersect = arcpy.analysis.Intersect([modal_composite_05a, urban_zoning_dissolve], intersect_output)

print("add length_urban field...")
#add length field if it doesnt already exist
list_fields = arcpy.ListFields(modal_zoning_intersect)
field_names = [i.name for i in list_fields]

if "length_urban" in field_names:
    print("length_urban field already exists")
else:
    print("adding length_urban field...")
    arcpy.AddField_management(modal_zoning_intersect,field_name="length_urban",field_type="FLOAT")
    print("done")

#calc field from shape.length with update cursor
print("calc length_urban field from shape.length...")
with arcpy.da.UpdateCursor(modal_zoning_intersect,["Shape_Length","length_urban"]) as cursor:
    for row in cursor:
        try:
            row[1] = row[0]        
            cursor.updateRow(row)    
        except ValueError as error:
            print(error)
print("done")

print("summarize intersect table by SegmentID...")
#run frequency on output and sum length fields to normalize data by SegmentID
freq_output = os.path.join(scratch_gdb_path, "freq_output")

freq = arcpy.analysis.Frequency(modal_zoning_intersect, freq_output, "SEGMENTID", "length_urban")

#join intersect freq result back to modal comp

print("Join field to modal_comp on SegmentID...")
join_target = modal_composite_05a
join_target_field = "SEGMENTID"
join_table = freq
join_table_field = "SEGMENTID"
join_fields = ["length_urban"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")

print("add new field for urban length percentage...")
#add new field for urban length % if it doesn't already exist
list_fields = arcpy.ListFields(modal_composite_05a)
field_names = [i.name for i in list_fields]

if "length_urban_percent" in field_names:
    print("length_urban_percent field already exists")
else:
    print("adding length_urban_percent field...")
    arcpy.AddField_management(modal_composite_05a,field_name="length_urban_percent",field_type="FLOAT", field_alias = "Urban % by length")
    print("done")

#calculate line percentages based on length differences
#calc field from shape.length with update cursor

print("calc urban length percent field...")
with arcpy.da.UpdateCursor(modal_composite_05a,["length_urban","Shape_Length","length_urban_percent","SEGMENTID"]) as cursor:
    for row in cursor:
        try:
            if row[0] is not None:
                row[2] = row[0]/row[1]  
                print("SegmentID: {0}, Length: {1}, Urban Length: {2}, Urban Percent: {3}".format(row[3], row[1], row[0], row[2]))
                
            cursor.updateRow(row)    
        except ValueError as error:
            print(error)
                       
            
##Compute Ag zoning line percentage metric

#select appropriate zoning categories, using the following ZONE_CLASS values
# AG-1 -
# AG-2 -


print("create ag zoning layer...")
ag_zoning = arcpy.management.MakeFeatureLayer(zoning, "ag_zoning", where_clause='"ZONE_CLASS" IN (\'AG-1\', \'AG-2\')' )

print("dissolve zoning...")
ag_dissolve_output = os.path.join(scratch_gdb_path, "ag_zoning_dissolve")

ag_zoning_dissolve = arcpy.management.Dissolve(ag_zoning, ag_dissolve_output)

print("intersect ag zoning w/ modal_comp...")
# intersect modal comp w/ selected zoning features
ag_intersect_output = os.path.join(scratch_gdb_path, "modal_ag_zoning_intersect")

modal_ag_zoning_intersect = arcpy.analysis.Intersect([modal_composite_05a, ag_zoning_dissolve], ag_intersect_output)

print("add length_ag field...")
#add length field if it doesnt already exist
list_fields = arcpy.ListFields(modal_ag_zoning_intersect)
field_names = [i.name for i in list_fields]

if "length_ag" in field_names:
    print("length_ag field already exists")
else:
    print("adding length_ag field...")
    arcpy.AddField_management(modal_ag_zoning_intersect,field_name="length_ag",field_type="FLOAT",field_alias = "Ag length")
    print("done")

#calc field from shape.length with update cursor
print("calc length_ag field from shape.length...")
with arcpy.da.UpdateCursor(modal_ag_zoning_intersect,["Shape_Length","length_ag"]) as cursor:
    for row in cursor:
        try:
            row[1] = row[0]        
            cursor.updateRow(row)    
        except ValueError as error:
            print(error)
print("done")

print("summarize intersect table by SegmentID...")
#run frequency on output and sum length fields to normalize data by SegmentID
ag_freq_output = os.path.join(scratch_gdb_path, "ag_freq_output")

ag_freq = arcpy.analysis.Frequency(modal_ag_zoning_intersect, ag_freq_output, "SEGMENTID", "length_ag")

#join intersect freq result back to modal comp

print("Join field to modal_comp on SegmentID...")
join_target = modal_composite_05a
join_target_field = "SEGMENTID"
join_table = ag_freq
join_table_field = "SEGMENTID"
join_fields = ["length_ag"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")

#add new field for urban length % if it doesn't already exist

print("add new field for Ag length percentage...")

list_fields = arcpy.ListFields(modal_composite_05a)
field_names = [i.name for i in list_fields]

if "length_ag_percent" in field_names:
    print("length_ag_percent field already exists")
else:
    print("adding length_ag_percent field...")
    arcpy.AddField_management(modal_composite_05a,field_name="length_ag_percent",field_type="FLOAT", field_alias = "Ag % by length")
    print("done")

#calculate line percentages based on length differences
#calc field from shape.length with update cursor
print("calc ag length percent field...")
with arcpy.da.UpdateCursor(modal_composite_05a,["length_ag","Shape_Length","length_ag_percent","SEGMENTID"]) as cursor:
    for row in cursor:
        try:
            if row[0] is not None:
                row[2] = row[0]/row[1]  
                print("SegmentID: {0}, Length: {1}, Ag Length: {2}, Ag Percent: {3}".format(row[3], row[1], row[0], row[2]))
                
            cursor.updateRow(row)    
        except ValueError as error:
            print(error)
            
            

            
            
print("processing complete")

In [ ]:
#CSDM Street Typology Processing

#Set CSDM analysis fields and filters

#fields to expose to update cursor
cs_fields = ["AB_LANE",               #0
             "BA_LANE",               #1
             "lanes_assumed",         #2
             "AB_SPEED",              #3
             "BA_SPEED",              #4
             "mean_row_2020",         #5
             "median_type",           #6
             "median_width",          #7
             "TOT_FLOW_D",            #8
             "CS_type",               #9
             "SEGMENTID",             #10
             "FULLNAME",              #11
             "length_urban_percent" , #12     
             "length_ag_percent",     #13
             "aadt",                  #14
             "SPD_LIMIT"]             #15

count_blvd = 0
count_ave = 0
count_main = 0
count_major_st = 0
count_st = 0
count_lane = 0
count_mall = 0
count_rural = 0
count_scenic = 0
count_error = 0

#"{0},{1}".format(round(float([AB_SPEED]),0),round(float([BA_SPEED]),0))



print("Setting Street Types....")
count = 0
with arcpy.da.UpdateCursor(modal_composite, cs_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #Set Boulevards
            if (((row[0] is not None) and (row[1] is not None) and row[0] + row[1] >= 4) and #lanes >=4                            
               ((row[15] is not None) and row[15] > 35) or row[6] == "R"): #Speed Limit >35
                
                count_blvd += 1
                row[9] = "Boulevard"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                      
            #Set Main Streets
            elif ((row[15] is not None) and (row[15] <= 35) and #Speed Limit <=35
                  (row[14] is not None and row[14] >= 10000) and #AADT > 10000
                  (row[12] is not None and row[12] > .5)): #Urban Percent > 50%
                
                count_main += 1
                row[9] = "Main Street"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                
            #Set Avenues    
            elif ((row[15] is not None) and (row[15] <= 35) and #Speed Limit <=35
                  (row[14] is not None and row[14] >= 10000) and #AADT > 10000
                  ((row[12] is not None and row[12] <= .5) or row[12] is None)): #Urban Percent is None or less than 50%
                
                count_ave += 1
                row[9] = "Avenue"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                    
            #Set Malls - hardcoded based on Segment Ids for Dukes Ln, Fort St Mall, S. Hotel St, Kekaulike St.
            elif (row[10] in (18914, 18967, 18984, 
                              19033, 19090, 19114, 
                              19148, 19152, 19233, 
                              19234, 19271, 19272, 
                              19306, 19319, 19333, 
                              19357, 19358, 19411, 
                              19456)):
                count_mall += 1
                row[9] = "Mall"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
            
            #Set Lanes & Alleys - ROW < 30 and has 2 lanes max and does not have TransCAD listing
            elif ((row[5] is not None and row[5] < 30) and #width < 30
                  (((row[0] is not None) and (row[1] is not None) and (row[0] + row[1] <=2)) or row[0] is None)): #2 lanes max or TransCAD is null 
                count_lane += 1
                row[9] = "Lane/Alley"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
            
            #Set Rural Roads - Rural % > 50 and does not have TransCAD or AADT data
            elif (row[13] is not None and row[13] > .5 and (row[0] is None) and (row[14] is None)):
                count_rural += 1
                row[9] = "Rural Road"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                
            #Set Scenic Byway's - hardcoded
            elif (row[10] in (18589,18648,23714,
                              33873,16814,33532,
                              33533,33534,33535,
                              33536,33537,33538)):
                count_scenic += 1
                row[9] = "Scenic Byway"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                  
            #Set Major Street    
            elif ((row[15] is not None) and (row[15] <= 35) and #Speed Limit <=35
                  ((row[0] is not None) and (row[1] is not None) and row[0] + row[1] <= 2) and #lanes >=4   
                  (row[14] is not None and row[14] >= 4000 and row[14] <= 10000)): #AADT > 10000 
                
                count_major_st += 1
                row[9] = "Major Street"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
                    
                    
            #Set Everything else to Street
            else:
                count_st += 1
                row[9] = "Street"
                #print("Segment ID: {0}, Name: {1}, Type: {2}".format(row[10],row[11],row[9]))
            
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)
            count_error += 1
            
print("Type - Boulevard: {}".format(count_blvd))
print("Type - Avenue: {}".format(count_ave))
print("Type - Main: {}".format(count_main))
print("Type - Mall: {}".format(count_mall))
print("Type - Lane: {}".format(count_lane))
print("Type - Rural: {}".format(count_rural))
print("Type - Scenic Byway: {}".format(count_scenic))
print("Type - Major Street: {}".format(count_major_st))
print("Type - Street: {}".format(count_st))
print("Types set for {} records".format((count_blvd + count_ave + count_main + count_mall + count_lane + count_rural + count_scenic + count_st)))

print("I am Error: {}x".format(count_error))

print("Typology Assignments complete.")


In [ ]:
#reset all CS Street Types to Null

with arcpy.da.UpdateCursor(modal_composite,"CS_type") as cursor:
    for row in cursor:
        #print("row")
        try:
            row[0] = None
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)
            
print("All CS Types reset to None")
          

In [ ]:
#create num fields for modal width calcualtions

#copy input dataset while testing functions
print("copying modal input for processing...")
modal_processing_temp = arcpy.CopyFeatures_management(modal_composite_04,"modal_composite_04a")
print("done")

#add field for assumed lanes for streets that do not have any lane data from TransCAD
arcpy.AddField_management(modal_composite_05a,field_name="lanes_assumed",field_type="SHORT",field_alias = "Lanes (assumed)")

#store field, field alias in dict
new_fields = {"modal_width_ped_exist":  "Modal Width - Ped (Existing)",
              "modal_width_ped_prop":   "Modal Width - Ped (Proposed)",
              "modal_width_bike_exist": "Modal Width - Bike (Existing)",
              "modal_width_bike_prop":  "Modal Width - Bike (Proposed)",
              "modal_width_auto_exist": "Modal Width - Auto (Existing)",
              "modal_width_auto_prop":  "Modal Width - Auto (Proposed)",
              "modal_width_bus_exist":  "Modal Width - Bus (Existing)",
              "modal_width_bus_prop":   "Modal Width - Bus (Proposed)",
              "modal_width_park_exist": "Modal Width - Parking (Existing)",
              "modal_width_park_prop":  "Modal Width - Parking (Proposed)",
              "modal_width_medians":    "Modal Width - Medians",
              "modal_width_max":        "Modal Width - Total Unconstrained"}

#loop through dict and add fields
for key,value in new_fields.items():
    print("Adding field {0}".format(key))
    arcpy.AddField_management(modal_composite_05a,field_name=key,field_type="SHORT", field_alias = value)
    

          
print("Field additions complete")


In [ ]:
#fix bike proposed missing fac_type field

bike_prop_intr = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb\Bike_Proposed_RCL_intersect"

bike_prop_alias = {
    "Fac_Type":"Facility Type (Bike Proposed)"
}

#rename bike proposed field
for key, value in bike_prop_alias.items():
    new_field_name = str(key)+"_BP"
    print("assigning alias {} to field {}, field renamed to {}".format(value,key,new_field_name))
    arcpy.AlterField_management(bike_prop_intr,key,new_field_name, value)
    
    
#join Bike Proposed intersect results
print("Joining Bike Proposed fields...")
join_target = modal_composite_05a
join_target_field = "SEGMENTID"
join_table = bike_prop_intr
join_table_field = "SEGMENTID"
join_fields = ["Fac_Type_BP"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")
    
print("Done")



In [ ]:
#join fields from median classification
print("Joining Median fields...")
join_target = modal_composite_05a
join_target_field = "SEGMENTID"
join_table = medians
join_table_field = "SEGMENTID"
join_fields = ["median_type","median_width","progression"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")
    
print("Done")


In [ ]:
#Temp DF for data analysis
temp_df = pd.DataFrame.spatial.from_featureclass(HERE)

temp_df_fields = temp_df.columns.values.tolist()

temp_df_fields.sort()

for i in temp_df_fields:
    print(i)

In [ ]:
#Join fieds from HMPS
print("Joining HPMS fields...")
join_target = modal_composite_05_2
join_target_field = "SEGMENTID"
join_table = HPMS
join_table_field = "SEGMENTID"
join_fields = ["aadt","route_id","funsystem", "funsysdesc","factype","factypedes","thrulanes"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join HPMS Fields complete.")

#Join fields from HERE
print("Joining HERE fields...")
join_target = modal_composite_05_2
join_target_field = "SEGMENTID"
join_table = HERE
join_table_field = "SEGMENTID"
join_fields = ["SPEED","SPD_LIMIT","MIN_SPD_LIMIT", "MAX_SPD_LIMIT"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join HERE Fields complete.")
    
print("Done")


In [ ]:
#update field alias for HPMS and HERE fields

update_alias = {
    "aadt":"HPMS - Annual Average Daily Traffic, Two Directions",
    "route_id":"HPMS - Route Number",
    "funsystem":"HPMS - Functional System Code",
    "funsysdesc":"HPMS - Functional System Code Description",
    "factype":"HPMS - Facility Type Code",
    "factypedes":"HPMS - Facility Type Code Description",
    "thrulanes":"HPMS - Number of Through Lanes, Two Directions",
    "SPD_LIMIT":"Speed Limit (HERE)",
    "MIN_SPD_LIMIT":"Speed Limit (HERE - low)",
    "MAX_SPD_LIMIT":"Speed Limit (HERE - high)"
}

#rename bike proposed field
for key, value in update_alias.items():
    new_field_name = str(key)+"_BP"
    print("assigning alias {0} to field {1}".format(value,key))
    arcpy.AlterField_management(modal_composite_05_2,key,new_field_alias=value)
    
print("alias assignment done")

### Calculate Modal Widths for Auto Traffic

- Calculate existing modal with for auto traffic using lane count x average lane width into `modal_width_auto_exist`. 

- If VOC Max from TransCAD 2040 > 1.0, assume two-way streets have two additonal lanes added (22' additional width), and one-way streets have one additional lane added (11' additonal width). Proposed width is calcualted into `modal_width_auto_prop`

- If VOC Max from TransCAD 2040 is <1.0, existing width is calculted into proposed width


**Assumptions, Notes, and Parameters**
1. Average lane width parameter =  11' wide.
2. Street segments with no lane data have the following additional assumptions applied, in order of implementation: 
    1. If `ONEWAY = 0` (street is two-way) and `mean_row < 100` assume 2 lanes total for line segment. Adjust with `param_two_way`
    2. If `ONEWAY = 0` (street is two-way) and `mean_row > 100` disregard (segments are typically edge cases)
    3. If `ONEWAY IN (1,2)` (street is one-way) and `mean_row < 75` assume 1 lane total for line segment. Adjust with `param_one_way`
    4. If `ONEWAY IN (1,2)` (street is one-way) and `mean_row > 75` disregard (segments are typically edge cases)
3. All street segments that have lane data, but do not have TransCAD VOC Max data have `modal_width_auto_exist` calculated into `modal_width_auto_prop`
4. Remaining street segments with no lane data, no mean_row calcualtion, and no TransCAD data, but have `CLASS = A41` are assumed to be 2 lanes, 22' width (regardless of oneway/two-way)
4. All street segments that were unable to have mean_row calculated, and do not have lane data are ignored


In [ ]:
#Conditional Re-Write
#PARAMETERS
param_one_way = 75.0
param_two_way = 100.0

#fields to expose to update cursor
VOC_fields = ["AB_VOC_EA_1", #0
              "BA_VOC_EA_1", #1
              "AB_VOC_AM_1", #2
              "BA_VOC_AM_1", #3
              "AB_VOC_MD_1", #4
              "AB_VOC_MD_1", #5
              "AB_VOC_PM_1", #6
              "BA_VOC_PM_1", #7
              "AB_VOC_EV_1", #8
              "BA_VOC_EV_1", #9
              "ONEWAY",      #10
              "modal_width_auto_exist", #11
              "modal_width_auto_prop",  #12
              "AB_LANE",     #13
              "BA_LANE",     #14
              "SEGMENTID",   #15
              "ln_exist_num",#16
              "AB_VOC_MAX",  #17
              "BA_VOC_MAX",  #18 
              "mean_row_2020", #19
              "lanes_assumed", #20,
              "CLASS"         #21
             ]

print("Update rows for Traffic widths...")
count = 0
count_01_01 = 0
count_01_02 = 0
count_01_03 = 0
count_01_04 = 0
count_02_01 = 0
count_02_02 = 0
count_02_03 = 0
count_02_04 = 0
count_03_01 = 0
count_03_02 = 0
count_03_03 = 0
count_03_04 = 0
count_03_05 = 0
voc_above = 0
voc_below = 0
voc_null = 0
err_count = 0

with arcpy.da.UpdateCursor(modal_composite, VOC_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #CONDITIONAL GROUP 01
            
            #VOC > 1.0
            #One Way Streets
            #TransCAD Lanes
#             if (not no(row[17] is not None and row[18] is not None) and (row[17] >= 1.0 or row[18] >= 1.0) and #VOC Max > 1.0
#                 ((row[13] is not None or row[14] is not None) and row[10] in [1,2])):
            
            if ((row[17] is not None and row[18] is not None) and (row[17] >= 1.0 or row[18] >= 1.0) and #VOC Max > 1.0
                ((row[13] is not None or row[14] is not None) and row[10] in [1,2])):
                
                #take max of the two lane values and multiply by 11
                current_width = (max(row[13],row[14]))*11
                expanded_width = ((max(row[13],row[14])+1)*11)
            
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2}".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                voc_above +=1 
                count_01_01 +=1
                cond = "01_01"
            
            #VOC > 1.0
            #One Way Streets
            #ASSUMED Lanes
            elif ((row[17] is not None and row[18] is not None) and (row[17] >= 1.0 or row[18] >= 1.0) and 
                  ((row[13] is None and row[14] is None and (row[10] in [1,2]) and (row[19] is not None) and row[19] < param_one_way))):
                
                current_width = 11
                expanded_width = 22
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 1
                voc_above +=1 
                count_01_02 +=1
                cond = "01_02"
                
                
            #VOC > 1.0
            #Two Way Streets
            #TransCAD Lanes
            elif ((row[17] is not None and row[18] is not None) and (row[17] >= 1.0 or row[18] >= 1.0) and
                  (row[13] is not None and row[14] is not None and (row[10] == 0))):
                
                current_width = (row[13] + row[14])*11
                expanded_width = ((row[13] + row[14]+2)*11)
                #print("Two Way - VOC Max:{0}, Current:{1}, Proposed: {2}".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                voc_above +=1
                count_01_03 +=1
                cond = "01_03"
                
            #VOC > 1.0
            #Two Way Streets
            #ASSUMED 2 Lanes
            elif ((row[17] is not None and row[18] is not None) and (row[17] >= 1.0 or row[18] >= 1.0) and
                  (row[13] is None and row[14] is None and (row[10] == 0) and (row[19] is not None) and row[19] < param_two_way)):
                
                current_width = 22
                expanded_width = 44
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 2 lanes".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 2
                voc_above +=1
                count_01_04 +=1
                cond = "01_04"
                
            
            #CONDITIONAL GROUP 02
            
            #VOC < 1.0
            #One Way Streets
            #TransCAD Lanes
            elif (not None in (row[17], row[18], row[13], row[14]) and (row[17] < 1.0 and row[18] < 1.0)):
        
            
# #             elif ((row[17] is not None and row[18] is not None) and (row[17] < 1.0 and row[18] < 1.0) and
# #                   (row[13] is not None and row[14] is not None) and row[10] in [1,2]):
                
                #current_width = (max(row[13],row[14]))*11
                current_width = 50
                expanded_width = current_width
                #print("One Way - VOC Max <1.0, Current:{0}, Proposed: {1} - {2}-{3}".format(current_width,expanded_width, row[17], row[18]))
                row[11] = current_width
                row[12] = expanded_width
                count_02_01 +=1
                voc_below += 1
                cond = "02_01"
                
            
            #VOC < 1.0
            #One Way Streets
            #ASSUMED One Lane
            elif ((row[17] is not None and row[18] is not None) and (row[17] < 1.0 and row[18] < 1.0) and
                  (row[13] is None and row[14] is None and row[10] in [1,2] and (row[18] is not None) and row[18] < param_one_way)):
                
                current_width = 11
                expanded_width = current_width
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 1
                count_02_02 +=1
                voc_below += 1
                cond = "02_02"
                
            #VOC < 1.0
            #Two Way Streets
            #TransCAD Lanes
            elif (not None in (row[17], row[18], row[13], row[14]) and (row[17] < 1.0 and row[18] < 1.0) and 
                  (row[10] == 0)):
            
            
#             elif ((row[17] is not None and row[18] is not None) and ((row[17] < 1.0) or (row[18] < 1.0)) and 
#                   (row[13] is not None and row[14] is not None and (row[10] == 0))):
                
                current_width = (row[13] + row[14])*11
                expanded_width = current_width
                #print("Two Way - VOC Max <1.0, Current:{0}, Proposed: {1}".format(current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                count_02_03 +=1
                voc_below += 1
                cond = "02_03"
                
            #VOC < 1.0
            #Two Way Streets
            #ASSUMED Two Lanes
            elif ((row[17] is not None and row[18] is not None) and (row[17] < 1.0 and row[18] < 1.0) and
                  (row[13] is None and row[14] is None and (row[10] == 0) and (row[19] is not None) and row[19] < param_two_way)):
                
                current_width = 22
                expanded_width = current_width
                #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 2 lanes".format(VOC_max,current_width,expanded_width))
                row[11] = current_width
                row[12] = expanded_width
                row[20] = 2
                count_02_04 +=1
                voc_below += 1
                cond = "02_04"
                    
            #CONDITIONAL GROUP 03
            
            #VOC Null
            #One Way Street
            #TransCAD Lanes
            #note this most likely does not occur but was implemented to catch records if they do occur
            elif (None in (row[17],row[18]) and (not None in (row[13],row[14]) and row[10] in [1,2])):
                                                 
                #(row[13] is not None and row[10] in [1,2]) or (row[14] is not None and row[10] in [1,2])):
                
                current_width = (max(row[13],row[14]))*11
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A".format(current_width))
                row[11] = current_width
                row[12] = current_width
                count_03_01 +=1
                voc_null += 1
                cond = "03_01"
                
            #VOC Null
            #One Way Street
            #ASSUMED One Lane
            elif ((row[17] is None) or (row[18] is None) and
                  (row[13] is None and row[14] is None and row[10] in [1,2] and (row[19] is not None) and row[19] < param_one_way)):
                
                current_width = 11
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 1 lane".format(current_width))
                row[11] = current_width
                row[12] = current_width
                row[20] = 1
                count_03_02 +=1
                voc_null += 1
                cond = "03_02"
                
            #VOC Null
            #Two Way Street
            #TransCAD Lanes
            #note this most likely does not occur but was implemented to catch records if they do occur
            elif ((row[17] is None) or (row[18] is None) and
                  (row[13] is not None and row[14] is not None and (row[10] == 0))):
                
                current_width = (row[13] + row[14])*11
                #print("Two Way - VOC Max: NULL, Current:{0}, Proposed: N/A".format(current_width))
                row[11] = current_width
                row[12] = current_width
                count_03_03 +=1
                voc_null += 1
                cond = "03_03"
                
            #VOC Null
            #Two Way Street
            #ASSUMED Two Lanes
            elif ((row[17] is None) or (row[18] is None) and
                  (row[13] is None and row[14] is None and (row[10] == 0) and (row[19] is not None) and row[19] < param_two_way)):
                
                current_width = 22
                #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 2 lanes".format(current_width))
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                count_03_04 +=1
                voc_null += 1
                cond = "03_04"
                
            #VOC Null
            #Mean ROW 2020 Null
            #Street Class = A41
            #ASSUMED Two Lanes, ASSUME Modal Width (current and future) = 22'
            elif ((row[17] is None) or (row[18] is None) and
                  ((row[13] is None and row[14] is None and row[19] is None) and row[21] == 'A41')):
                
                current_width = 22
                #print("A41 Assumption - VOC Max: NULL, Current:NULL, Proposed: N/A Assumed 2 lanes".format(current_width))
                row[11] = current_width
                row[12] = current_width
                row[20] = 2
                count_03_05 +=1
                voc_null += 1
                cond = "03_05"
                                          
            #cursor.updateRow(row)
            count = count + 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))
            
            
print("Done - processed {0} records, {1} errors".format(count, err_count))
print("01-01 VOC >1.0 - One Way - TransCad Lanes: {}".format(count_01_01))
print("01-02 VOC >1.0 - One Way - Assumed Lanes: {}".format(count_01_02))
print("01-03 VOC >1.0 - Two Way - TransCad Lanes: {}".format(count_01_03))
print("01-04 VOC >1.0 - Twy Way - Assumed Lanes: {}".format(count_01_04))

print("02-01 VOC <1.0 - One Way - TransCad Lanes: {}".format(count_02_01))
print("02-02 VOC <1.0 - One Way - Assumed Lanes: {}".format(count_02_02))
print("02-03 VOC <1.0 - Two Way - TransCad Lanes: {}".format(count_02_03))
print("02-04 VOC <1.0 - Two Way - Assumed Lanes: {}".format(count_02_04))

print("03-01 VOC NULL - One Way - TransCad Lanes: {}".format(count_03_01))
print("03-02 VOC NULL - One Way - Assumed Lanes: {}".format(count_03_02))
print("03-03 VOC NULL - Two Way - TransCad Lanes: {}".format(count_03_03))
print("03-04 VOC NULL - Two Way - Assumed Lanes: {}".format(count_03_04))

print("03-05 A41 NULL -   N/A   - Assumed Lanes: {}".format(count_03_05))

print("VOC Above: {}".format(voc_above))
print("VOC Below: {}".format(voc_below))
print("VOC Null: {}".format(voc_null))
print("Modal Widths - Traffic complete.")


In [ ]:
lanes = 1
lane_width = (lanes-2)*10 + 22

expanded_width = (lanes-1)*10 + 22

print(lane_width)
print(expanded_width)

In [ ]:

##OLD
#PARAMETERS
param_one_way = 75.0
param_two_way = 100.0

#fields to expose to update cursor
VOC_fields = ["AB_VOC_EA_1", #0
              "BA_VOC_EA_1", #1
              "AB_VOC_AM_1", #2
              "BA_VOC_AM_1", #3
              "AB_VOC_MD_1", #4
              "AB_VOC_MD_1", #5
              "AB_VOC_PM_1", #6
              "BA_VOC_PM_1", #7
              "AB_VOC_EV_1", #8
              "BA_VOC_EV_1", #9
              "ONEWAY",      #10
              "modal_width_auto_exist", #11
              "modal_width_auto_prop",  #12
              "AB_LANE",     #13
              "BA_LANE",     #14
              "SEGMENTID",   #15
              "ln_exist_num",#16
              "AB_VOC_MAX",  #17
              "BA_VOC_MAX",  #18 
              "mean_row_2020", #19
              "lanes_assumed", #20,
              "CLASS"         #21
             ]

print("Update rows for Traffic widths...")
count = 0
count_01_01 = 0
count_01_02 = 0
count_01_03 = 0
count_01_04 = 0
count_02_01 = 0
count_02_02 = 0
count_02_03 = 0
count_02_04 = 0
count_03_01 = 0
count_03_02 = 0
count_03_03 = 0
count_03_04 = 0
count_03_05 = 0
voc_above = 0
voc_below = 0
voc_null = 0
err_count = 0

with arcpy.da.UpdateCursor(modal_composite, VOC_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #AB_VOC_MAX > 1.0 or BA_VOC_MAX > 1.0         
            if ((row[17] is not None and row[17] >= 1.0) or (row[18] is not None and row[18]>=1.0)):
                VOC_max = max(row[17],row[18])
                voc_above +=1
                #print("Conditional 01 - VOC > 1.0...")
                
                #CONDITIONAL LEVEL 01-01
                # one way street width calc for VOC > 1.0 with lane values from TransCAD              
                if ((row[13] is not None) and row[10] in [1,2] or
                    ((row[14] is not None) and row[10] in [1,2])):
                    
                    lanes = max(row[13],row[14])
                    if lanes == 1:
                        current_width = lanes*11
                        expanded_width = (lanes+1)*11 
                    elif lanes >1:
                        current_width = (lanes-2)*10 + 22
                        expanded_width = (lanes-1)*10 + 22
                    
                    
                    #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2}".format(VOC_max,current_width,expanded_width))
                    row[11] = current_width
                    row[12] = expanded_width
                    count_01_01 +=1
                    
                #CONDITIONAL LEVEL 01-02
                #ASSUMED LANES = 1
                # one way street width calc for VOC > 1.0 with NULL lanes values
                elif (row[13] is None and row[14] is None and (row[10] in [1,2]) and (row[19] is not None) and row[19] < param_one_way):
                    #print("01-02 pass")
                    current_width = 11
                    expanded_width = 22
                    #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                    row[11] = current_width
                    row[12] = expanded_width
                    row[20] = 1
                    count_01_02 +=1
                
                #CONDITIONAL LEVEL 01-03    
                # two way street width calc for VOC > 1.0 with lane values from TransCAD
                elif (row[13] is not None and row[14] is not None and (row[10] == 0)):
                    #print("01-03 pass")
                    lanes = max(row[13],row[14])
                    if lanes == 1:
                        current_width = lanes*11
                        expanded_width = (lanes+1)*11 
                    elif lanes >1:
                        current_width = (lanes-2)*10 + 22
                        expanded_width = (lanes-1)*10 + 22

                    #print("Two Way - VOC Max:{0}, Current:{1}, Proposed: {2}".format(VOC_max,current_width,expanded_width))
                    row[11] = current_width
                    row[12] = expanded_width
                    count_01_03 +=1
                    
                #CONDITIONAL LEVEL 01-04
                #ASSUMED LANES = 2
                # two way street width calc for VOC > 1.0 with NULL lanes values
                elif (row[13] is None and row[14] is None and (row[10] == 0) and (row[19] is not None) and row[19] < param_two_way):
                    #print("01-04 pass")
                    current_width = 22
                    expanded_width = 42
                    #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 2 lanes".format(VOC_max,current_width,expanded_width))
                    row[11] = current_width
                    row[12] = expanded_width
                    row[20] = 2
                    count_01_04 +=1
                    
            #end CONDITIONAL LEVEL 01
            
            #CONDITIONAL LEVEL 02
            #process all records where VOC < 1.0
            elif (((row[17] is not None) and row[17] < 1.0) and ((row[18] is not None) and row[18]<1.0)):
                VOC_max = max(row[17],row[18])
                voc_below += 1
                #print("Conditional 02 - VOC < 1.0...")
              
                #CONDITIONAL LEVEL 02-01
                # one way street width calc for VOC < 1.0 (not calculated) with lanes from TransCAD
                if ((row[13] is not None and row[10] in [1,2]) or
                    (row[14] is not None and row[10] in [1,2])):
                    #print("02-01 pass")
                    lanes = max(row[13],row[14])
                    if lanes == 1:
                        current_width = lanes*11
                    elif lanes >1:
                        current_width = (lanes-2)*10 + 22
                    #print("One Way - VOC Max <1.0, Current:{0}, Proposed: {1}".format(current_width,expanded_width))
                    row[11] = current_width
                    row[12] = current_width
                    count_02_01 +=1
                    
                             
                #CONDITIONAL LEVEL 02-02
                #ASSUMED LANES = 1
                # one way street width calc for VOC < 1.0 with NULL lanes values
                elif (row[13] is None and row[14] is None and row[10] in [1,2] and (row[18] is not None) and row[18] < param_one_way):
                    #print("02-02 pass")
                    current_width = 11
                    #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 1 lane".format(VOC_max,current_width,expanded_width))
                    row[11] = current_width
                    row[12] = current_width
                    row[20] = 1
                    count_02_02 +=1

                #CONDITIONAL LEVEL 02-03
                # two way stret width calc for VOC < 1.0 (not calculated) with lanes from TransCAD      
                elif (row[13] is not None and row[14] is not None and (row[10] == 0)):
                    #print("02-03 pass")
                    lanes = max(row[13],row[14])
                    if lanes == 1:
                        current_width = lanes*11
                    elif lanes >1:
                        current_width = (lanes-2)*10 + 22
                    #print("Two Way - VOC Max <1.0, Current:{0}, Proposed: {1}".format(current_width,expanded_width))
                    row[11] = current_width
                    row[12] = current_width
                    count_02_03 +=1
                    
                #CONDITIONAL LEVEL 02-04
                #ASSUMED LANES = 2
                # two way street width calc for VOC < 1.0 with NULL lanes values
                elif (row[13] is None and row[14] is None and (row[10] == 0) and (row[19] is not None) and row[19] < param_two_way):        
                    #print("02-04 pass")
                    current_width = 22
                    #print("One Way - VOC Max:{0}, Current:{1}, Proposed: {2} Assumed 2 lanes".format(VOC_max,current_width,expanded_width))
                    row[11] = current_width
                    row[12] = current_width
                    row[20] = 2
                    count_02_04 +=1
                    
            #CONDITIONAL LEVEL 03
            #process records with no TransCAD VOC Data (set current widths only)
            elif ((row[17] is None) or (row[18] is None)):
                #print("Conditional 03 - no TransCAD VOC data...")
                voc_null += 1
                
                #CONDITIONAL LEVEL 03-01
                # one way street width calc for lanes from TransCAD
                if ((row[13] is not None and row[10] in [1,2]) or
                    (row[14] is not None and row[10] in [1,2])):
                    #print("03-01 pass")
                    lanes = max(row[13],row[14])
                    if lanes == 1:
                        current_width = lanes*11
                    elif lanes >1:
                        current_width = (lanes-2)*10 + 22
                    #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A".format(current_width))
                    row[11] = current_width
                    row[12] = current_width
                    count_03_01 +=1
                              
                #CONDITIONAL LEVEL 03-02
                #ASSUMED LANES = 1
                # one way street width calc for NULL lanes values
                elif (row[13] is None and row[14] is None and row[10] in [1,2] and (row[19] is not None) and row[19] < param_one_way):
                    #print("03-02 pass")
                    current_width = 11
                    #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 1 lane".format(current_width))
                    row[11] = current_width
                    row[12] = current_width
                    row[20] = 1
                    count_03_02 +=1

                #CONDITIONAL LEVEL 03-03
                # two way stret width calc for lanes from TransCAD      
                elif (row[13] is not None and row[14] is not None and (row[10] == 0)):
                    #print("03-03 pass")
                    lanes = max(row[13],row[14])
                    if lanes == 1:
                        current_width = lanes*11
                    elif lanes >1:
                        current_width = (lanes-2)*10 + 22
                    #print("Two Way - VOC Max: NULL, Current:{0}, Proposed: N/A".format(current_width))
                    row[11] = current_width
                    row[12] = current_width
                    count_03_03 +=1
                    
                #CONDITIONAL LEVEL 03-04
                #ASSUMED LANES = 2
                # two way street width calc for NULL lanes values
                elif (row[13] is None and row[14] is None and (row[10] == 0) and (row[19] is not None) and row[19] < param_two_way):
                    #print("03-04 pass")
                    current_width = 22
                    #print("One Way - VOC Max: NULL, Current:{0}, Proposed: N/A Assumed 2 lanes".format(current_width))
                    row[11] = current_width
                    row[12] = current_width
                    row[20] = 2
                    count_03_04 +=1 
                    
                #CONDITIONAL LEVEL 03-05
                #ASSUMED LANES = 2
                #process records with no TransCAD data, no mean_row calculation, where CLASS == 'A41'
                elif ((row[13] is None and row[14] is None and row[19] is None) and row[21] == 'A41'):
                    #print("03-05 pass")
                    current_width = 22
                    #print("A41 Assumption - VOC Max: NULL, Current:NULL, Proposed: N/A Assumed 2 lanes".format(current_width))
                    row[11] = current_width
                    row[12] = current_width
                    row[20] = 2
                    count_03_05 +=1 
                              
            cursor.updateRow(row)
            count = count + 1
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[15],error))
            
            
print("Done - processed {0} records, {1} errors".format(count, err_count))
print("01-01 VOC >1.0 - One Way - TransCad Lanes: {}".format(count_01_01))
print("01-02 VOC >1.0 - One Way - Assumed Lanes: {}".format(count_01_02))
print("01-03 VOC >1.0 - Two Way - TransCad Lanes: {}".format(count_01_03))
print("01-04 VOC >1.0 - Twy Way - Assumed Lanes: {}".format(count_01_04))

print("02-01 VOC <1.0 - One Way - TransCad Lanes: {}".format(count_02_01))
print("02-02 VOC <1.0 - One Way - Assumed Lanes: {}".format(count_02_02))
print("02-03 VOC <1.0 - Two Way - TransCad Lanes: {}".format(count_02_03))
print("02-04 VOC <1.0 - Two Way - Assumed Lanes: {}".format(count_02_04))

print("03-01 VOC NULL - One Way - TransCad Lanes: {}".format(count_03_01))
print("03-02 VOC NULL - One Way - Assumed Lanes: {}".format(count_03_02))
print("03-03 VOC NULL - Two Way - TransCad Lanes: {}".format(count_03_03))
print("03-04 VOC NULL - Two Way - Assumed Lanes: {}".format(count_03_04))

print("03-05 A41 NULL -   N/A   - Assumed Lanes: {}".format(count_03_05))

print("VOC Above: {}".format(voc_above))
print("VOC Below: {}".format(voc_below))
print("VOC Null: {}".format(voc_null))
print("Modal Widths - Traffic complete.")


### Calculate Modal Widths for Bike Facilities


- Existing width is calculated into `modal_width_bike_exist` and `modal_width_bike_prop`

- Proposed width is then updated into `modal_width_bike_prop` based on proposed and redevelopment bike plan attributes.

- If a give segment has a value for existing bike facilites, but no values for proposed facilites, existing width is calculted into proposed width.


**Assumptions, Notes, and Parameters**
1. "Protected Bike Lane", "Buffered Bike Lane" = 10'
2. "Bike Lane", "Climbing Lane", "Shoulder Bikeway" = 7'


In [ ]:
#fields to expose to update cursor for bike facilities
fields = ["Fac_Type_BE",            #0
          "modal_width_bike_exist", #1
          "Fac_Type_BP",            #2
          "Fac_Type_BR",            #3
          "modal_width_bike_prop"]  #4

#update bike existing width columns
print("Update rows for Bike widths..")
with arcpy.da.UpdateCursor(modal_composite_05a,fields,'NOT ("Fac_Type_BE" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike existing calculations
            if row[0] == "Protected Bike Lane" or row[0] == "Buffered Bike Lane":
                row[1] = 12
                row[4] = 12
                    
            elif (row[0] == "Bike Lane" or
                  row[0] == "Climbing Lane" or
                  row[0] == "Shoulder Bikeway" or
                  row[0] == "Shared Use Path" or
                  row[0] == "Shared Roadway"):
                row[1] = 7
                row[4] = 7
                    
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("bike existing width calculated\n")

#perform second pass w/ update cursor for bike proposed features (will over write previous calcs if there is an improvement)
#update bike proposed width columns
print("Update rows for Bike widths..")
with arcpy.da.UpdateCursor(modal_composite_05a,fields,'NOT ("Fac_Type_BP" IS NULL AND "Fac_Type_BR" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike proposed & redev calculations
            if (row[2] == "Protected Bike Lane" or
                  row[2] == "Buffered Bike Lane" or
                  row[3] == "Protected Bike Lane" or
                  row[3] == "Buffered Bike Lane"):
                row[4] = 12
                
            elif (row[2] == "Bike Lane" or
                  row[2] == "Climbing Lane" or
                  row[2] == "Shoulder Bikeway" or
                  row[2] == "Shared Use Path" or
                  row[2] == "Shared Roadway" or
                  row[3] == "Bike Lane" or
                  row[3] == "Climbing Lane" or
                  row[3] == "Shoulder Bikeway" or
                  row[3] == "Shared Use Path" or
                  row[3] == "Shared Roadway"):
                row[4] = 7
                
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("bike proposed width calculated\n")
print("Modal Widths - Bike complete.")

In [ ]:
#data analysis for medians
fields = ["SEGMENTID",
          "FULLNAME",
          "mean_row_2020",
          "NBRightP_1",
          "SBLeftPe_1",
          "FID_Ped_Improve_temp",
          "FID_Ped_Add_temp",
          "modal_width_ped_exist",
          "median_type",
          "median_width"]

fields_median = ["SEGMENTID",
          "FULLNAME",
          "mean_row_2020",
          "modal_width_medians",
          "median_type",
          "median_width"]

#two way streets, null lane count, row > 100
filt2 = ((~sdf["FID_Ped_Improve_temp"].isnull()) |
         (~sdf["FID_Ped_Add_temp"].isnull())
        )


filt3 = ((sdf["FID_Ped_Improve_temp"].isnull()) &
         (~sdf["FID_Ped_Add_temp"].isnull()) &
         (sdf["NBRightP_1"] == 0) &
         (sdf["SBLeftPe_1"] == 0)
        )
         
#one way streets, null lane count, row > 75
filt4 = ((sdf["AB_LANE"].isnull()) &
         (sdf["BA_LANE"].isnull()) &
         (sdf["ONEWAY"].isin([1,2])) &
         (sdf["mean_row_2020"]>75)
        )

#two way streets, null lane count, row > 100
filt_median = ~sdf["median_type"].isnull()

#pd.set_option('display.max_rows', 250)
sdf.loc[filt_median,fields_median].sort_values(by="FULLNAME")

In [ ]:
sdf2.loc[sdf2["median_type"]== "R","median_type"] = "Raised with Curb"
sdf2.loc[sdf2["median_type"]== "F","median_type"] = "Flush Two-Way Left Turn"
sdf2.loc[sdf2["median_type"]== "C","median_type"] = "Composite"
sdf2.loc[sdf2["median_type"]== "FB","median_type"] = "Flush with Barrier"

sdf2.loc[sdf2["median_width"]== "A","median_width"] = 5
sdf2.loc[sdf2["median_width"]== "B","median_width"] = 11
sdf2.loc[sdf2["median_width"]== "C","median_width"] = 17
sdf2.loc[sdf2["median_width"]== "D","median_width"] = 22

sdf2_filt = ((~sdf2["median_type"].isnull()) &
             (~sdf2["median_type"].isin(["Raised", "Flush", "Composite", "Flush with Barrier"]))
            )

sdf2.loc[~sdf2["median_type"].isnull(),["SEGMENTID", "median_type", "median_width"]]

In [ ]:
sdf_fields

### Calculate Modal Widths for Medians

- Median data was manually classified by reviewing street segments where `mean_row_2020 > 75`, then estimaing median type and median width using aerial imagery.
- Width estimates were then mapped into `modal_width_medians` (see assumptions below)

**Assumptions, Notes, and Parameters**
1. Median width esitmations were mapped to modal with values as follows:

|Median Width Estimate| Modal Width Value  |
|:--|:--|
| < 5' |5'  |
| 5' - 10' |11' (increase by 1' to match avg. lane width)  |
| 15' - 20' |17'  |
| >20' |22'  |


In [ ]:
#Calculate Modal Width for Medians

# "A","median_width"] = 5
# "B","median_width"] = 11
# "C","median_width"] = 17
# "D","median_width"] = 22

#fields to expose to update cursor
fields = ["median_type",         #0
          "median_width",        #1
          "modal_width_medians"] #2

#update median width columns
print("Update rows for Medians widths..")
count = 0
with arcpy.da.UpdateCursor(modal_composite,fields,'NOT ("median_type" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        
        # 6/16/2021 - increase all values by 2
        
        try:
            if row[1] == "A":
                row[2] = 7
                count +=1
            elif row[1] == "B":
                row[2] = 13
                count +=1
            elif row[1] == "C":
                row[2] = 19
                count +=1
            elif row[1] == "D":
                row[2] = 14
                count +=1
                    
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)
            
print("median widths calculated for {} items".format(count))
print("Modal Widths - Medians complete.")

### Calculate Modal Widths for Ped Facilities

- Existing width is calculated into `modal_width_ped_exist` and `modal_width_ped_prop`

- Proposed width `modal_width_ped_prop` is then updated if a segment had no ped facilites, was not listed in ped facilites, or was listed in "Candidate Walkways" or "Candidate Upgrades", and the proposed width was greater than the existing width. (see assumptions below)


**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Any segment listed in `NewTier` == 4 is set to 8' if oneway, 16' if two-way for future widths.
3. The following steet segments types are set to 12' for `modal_width_ped_prop`
    1. Any street segment listed in "Candidate Walkways", and "Candidate Upgrades" datasets
    2. Any street segment listed in Pedestrian Sidewalk Inventory as having no facilites, but are not listed in "Candidate Walkways" or "Candidate Upgrades"
    3. Any street segment NOT listed in Pedestrian Sidewalk Inventory, AND not listed in "Candidate Walkways", AND not listed in "Candidate Upgrades", with `CLASS = A41`


In [ ]:
#calculate modal width for ped facilities

#assumptions based on ped data dictionary values for PED ZONE WIDTH
# greater than 10' = 12' DOMAIN 04
# 5'-10' (clear and obstruced types) = 8' DOMAIN 03, DOMAIN 02
# less than 5' = 4' DOMAIN 01

p1 = 12
p2 = 8
p3 = 4

#facilities combinations
#24' total
#20' total
#16' total
#12' total
#8' total
#4' total
# none on either side

#existing ped facilities

#expose fields to update cursor
ped_e_fields = ["SEGMENTID",             #0 - SegmentID
                "NBRightP_1",            #1 - Pedestrian Zone Width (NB)
                "SBLeftPe_1",            #2 - Pedestiran Zone Width (SB)
                "modal_width_ped_exist", #3 - Modal Width - Ped (Existing)
                "modal_width_ped_prop",  #4 - Modal Width - Ped (Proposed)
                "mean_row_2020",         #5 - Mean ROW Analysis
                "FID_Ped_Improve_temp",  #6 - Ped Improvements
                "FID_Ped_Add_temp",      #7 - Ped Additions
                "CLASS",                 #8 - Street Class (tiger)
                "NewTier",               #9 - Bus Transit - New Proposed Tier
                "ONEWAY",                #10 - Bus Transit - New Proposed Tier
                "NBRightP_7",            #11 - Pedestrian Buffer Roadway Buffer (NB)
                "SBLeftPe_6"]            #12 - Pedestrian Buffer Roadway Buffer (SB)


count = 0
err_count = 0

print("Update rows with Ped Zone widths..")

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields) as cursor:
# with arcpy.da.UpdateCursor(modal_composite_05a, ped_e_fields,'NOT ("NBRightP_1" IS NULL OR "SBLeftPe_1" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #24' total
            count +=1
            if row[1] == 4 and row[2] == 4:
                width = p1*2
                row[3] = width
                row[4] = width
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
        
            #20' total
            elif ((row[1] == 4 and row[2] in [2,3])or
                  (row[1] in [2,3] and row[2] == 4)):
                width = p1+p2
                row[3] = width
                row[4] = width
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))

            #16' total
            elif ((row[1] in [2,3]) and (row[2] in [2,3])):
                width = p2*2
                row[3] = width
                row[4] = width
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))

            #12' total
            elif ((row[1] == 1 and (row[2] in [2,3]))or
                  ((row[1] in [2,3]) and row[2] == 1)):
                width = p2+p3
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
            
            #8' total
            elif row[1] == 1 and row[2] == 1:
                width = p3*2
                row[3] = width
                row[4] = 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))

            #4' total
            elif ((row[1] == 1 and row[2] == 0) or
                  (row[1] == 0 and row[2] == 1)):
                width = p3
                row[3] = width
                row[4] = 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))


            # none on either side, set future width to 12'
            elif (row[1] == 0 and row[2] == 0):
                width = 0
                row[3] = width
                row[4] = 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                  
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Done - processed {0} records, {1} errors".format(count, err_count))


#assumptions based on ped data dictionary values for PED FURNITURE ZONE WIDTH
# greater than 4' = 6' DOMAIN 03
# 0' to 4' = 4' DOMAIN 02
# 0' = 0' DOMAIN 01

#add cursor pass for furniture zone

print("Update rows with Ped Furnitur widths..")

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            #12' total
            count +=1
            if row[11] == 3 and row[12] == 3:

                row[3] += 12
                row[4] += 12
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
        
            #10' total
            elif ((row[11] == 3 and row[12] == 2)or
                  (row[11] == 2 and row[12] == 3)):
                
                row[3] += 10
                row[4] += 10
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                
            #8' total
            elif (row[11] == 2 and row[12] == 2):
         
                row[3] += 8
                row[4] += 8
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))
                
            #4' total
            elif ((row[11] == 2 and row[12] == 1)or
                  (row[11] == 1 and row[12] == 2)):
        
                row[3] += 4
                row[4] += 4
                #print("SegmentID: {0}, NB Width: {1}, SB Width: {2}, Total Width:{3} ".format(row[0],row[1],row[2],width))


                  
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Done - processed {0} records, {1} errors".format(count, err_count))
                
                

#proposed ped facilities - use a second pass with update cursor to update proposed widths to 12' if they
# have segments listed in Ped improvement datasets, and the existing width is less than 12'
count = 0
err_count = 0

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields,'(NOT "FID_Ped_Improve_temp" IS NULL) OR (NOT "FID_Ped_Add_temp" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #only update proposed width if is less than the current width
            if ((row[4] is not None) and row[4] < 12):
                row[4] = 12
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
#proposed bus transit tier 4 - update width to 16' if twoway, 8' if oneway
count = 0
err_count = 0

with arcpy.da.UpdateCursor(modal_composite, ped_e_fields,'"NewTier" = 4') as cursor:
    for row in cursor:
        #print("New Tier = 4 Row")
        try:
            #only update proposed width if is less than the current width
            if (((row[10] is not None) and row[10] == 0) and
                ((row[4] is None) or (row[4] is not None) and row[4] < 16)):
                row[4] = 16
                #print("Tier4 two way")
                
            elif((row[10] is not None) and row[10] in [1,2] and
                 ((row[4] is None) or (row[4] is not None) and row[4] < 8)):
                row[4] = 8
                #print("Tier4 one way")             
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            

# perform final pass with update cursor to add proposed width to facilites that have no sidewalks
# and are not covered by sidewalk inventory, but are residential (class = 'A41')
count = 0
err_count = 0

#with arcpy.da.UpdateCursor(modal_composite_05a, ped_e_fields,'"NBRightP_1" IS NULL AND "FID_Ped_Improve_temp" IS NULL AND "FID_Ped_Add_temp" IS NULL AND "CLASS" = \'A41\' ') as cursor:
with arcpy.da.UpdateCursor(modal_composite, ped_e_fields,'"NBRightP_1" IS NULL AND "FID_Ped_Improve_temp" IS NULL AND "FID_Ped_Add_temp" IS NULL') as cursor:
    for row in cursor:
        #print("final pass")
        try:
            #only update proposed width if is less than the current width
            if (row[8] == 'A41' and ((row[4] is not None and row[4] < 12) or row[4] is None)):
            #if ((row[8] == 'A41') and (row[4] is None)):
                row[4] = 12   
                #print("pass 3 for A41")
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            

print("Modal Widths - Ped complete.") 

### Calculate Modal Widths for on-street Parking segments

- Existing width of 7' is calculated into `modal_width_park_exist` and `modal_width_park_prop` for the following segment types:
    - Roadway Buffer = "Yes" `NBRightP_7 == 1 OR SBLeftPed_6 == 1`
    - Pedestrian Zone Intrusion = "Yes" `NBRightP_4 == 1 or SBLeftPed_4`
    
- If a segment is not listed in the Pedestrian Plan Sidewalk inventory, and it's street type is residential (A41), and mean_row_2020 - modal_width_auto_exist > 22, on street parking is assumed on both sides.




**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Street segments listed in "Candidate Walkways", and "Candidate Upgrades" datasets are assumed to be 6' sidewalk on both sides, (12' total)
4. Street segmnents listed in Pedestrian Sidewalk Inventory having no facilites, but are not listed in "Candidate Walkways", or "Candidate Upgrades" and are PLACEHOLDER are assumed to be upgraded to one 6' sidewalk. 6' total.


In [ ]:
#calculate modal width for street segements with on-street parking facilities


#expose fields to update cursor
parking_fields = ["SEGMENTID",              #0 - SegmentID
                  "NBRightP_7",             #1 - Road Buffer (NB)
                  "SBLeftPe_6",             #2 - Road Buffer (SB)
                  "NBRightP_4",             #3 - Ped Intrusion (NB)
                  "SBLeftPe_4",             #4 - Ped Intrusion (SB)
                  "modal_width_park_exist", #5 - Modal Width - Park (Existing)
                  "modal_width_park_prop",  #6 - Modal Width - Park (Proposed)
                  "mean_row_2020",          #7 - Mean ROW Analysis
                  "modal_width_auto_exist", #8 - Modal Width - Auto (Existing)
                  "CLASS"]                  #9 - Street Class (tiger)

park_ped_count = 0
park_assume_count = 0
err_count = 0

param = 22

print("Update rows for Parking widths..")

#calculate parking width for segments based on ped inventory facilities
with arcpy.da.UpdateCursor(modal_composite_05a, parking_fields,'"NBRightP_7" IS NOT NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[1] == 1 or row[2] == 1 or row[3] == 1 or row[4] == 1:
                row[5] = 8
                row[6] = 8
                park_ped_count +=1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            

#calculate parking width for segments not listed in ped inventory facilities
with arcpy.da.UpdateCursor(modal_composite_05a, parking_fields,'"NBRightP_7" IS NULL AND "CLASS" = \'A41\'') as cursor:
    for row in cursor:
        #print("row")
        try:
            if (row[7] is not None) and (row[8] is not None) and ((row[7] - row[8]) > param):
                row[5] = 8
                row[6] = 8
                park_assume_count +=1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Parking widths (ped) calculated for {0} items".format(park_ped_count))
print("Parking widths (assumed) calculated for {0} items".format(park_assume_count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Parking complete.")

### Calculate Modal Widths for Bus Transit segments

TEXT TO BE REPLACED

- Existing width of 7' is calculated into `modal_width_park_exist` and `modal_width_park_prop` for the following segment types:
    - Roadway Buffer = "Yes" `NBRightP_7 == 1 OR SBLeftPed_6 == 1`
    - Pedestrian Zone Intrusion = "Yes" `NBRightP_4 == 1 or SBLeftPed_4`
    
- If a segment is not listed in the Pedestrian Plan Sidewalk inventory, and it's street type is residential (A41), and mean_row_2020 - modal_width_auto_exist > 22, on street parking is assumed on both sides.




**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Street segments listed in "Candidate Walkways", and "Candidate Upgrades" datasets are assumed to be 6' sidewalk on both sides, (12' total)
4. Street segmnents listed in Pedestrian Sidewalk Inventory having no facilites, but are not listed in "Candidate Walkways", or "Candidate Upgrades" and are PLACEHOLDER are assumed to be upgraded to one 6' sidewalk. 6' total.


In [ ]:
#calculate modal width for bus facilities


#expose fields to update cursor
bus_fields = ["SEGMENTID",              #0 - SegmentID
              "NAME",                   #1 - Name
              "modal_width_bus_exist",  #2 - Modal Width - Bus (Existing)
              "modal_width_bus_prop",   #3 - Modal Width - Bus (Proposed)
              "mean_row_2020",          #4 - Mean ROW Analysis
              "CLASS",                  #5 - Street Class (tiger)
              "NewTier",                #6 - Bus Transit - New Tier
              "ONEWAY"]                 #7 - OneWay

bus_count = 0
new_count = 0
err_count = 0

#existing bus facilites by SegmentID (replace with Bus Plan data when available)

IDs_exist = (
    18813,18858,18867,18914,18968,18984,19024,19033,19068,19090,19140,19148,19148,19234,19272,
    19306,19320,19333,19357,19401,19441,19501,19595,19672,19758,19790,21993,22030,22049,22112,
    22163,22169,33146)

where_clause = '"SEGMENTID" IN {}'.format(IDs_exist)

print("Update rows for Bus widths..")
print("Calculating existing...")
#calculate existing Priority Bus facilities
with arcpy.da.UpdateCursor(modal_composite_05a, bus_fields, '"SEGMENTID" IN {}'.format(IDs_exist)) as cursor:   
    for row in cursor:
        #print("row")
        try:
            if row[1] == "HOTEL":
                row[2] = 24
                row[3] = 24
                bus_count += 1
            
            elif row[1] == "KALAKAUA" or row[1] == "KING":
                row[2] = 12
                row[3] = 12
                bus_count += 1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Existing calc done.")
            
#calculate proposed Tier4 Bus facilities
print("Calculating proposed...")
with arcpy.da.UpdateCursor(modal_composite_05a, bus_fields, '"NewTier" = 4') as cursor:
    for row in cursor:
        try:
            if ((row[7] is not None) and row[7] == 0):
                row[3] = 24
                new_count += 1
            elif((row[7] is not None) and row[7] in [1,2]):
                row[3] = 12
                new_count += 1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
print("Proposed calc done.") 
            
print("Bus widths - Existing -  calculated for {0} segments".format(bus_count))
print("Bus widths - New -       calculated for {0} segments".format(new_count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Bus complete.")

### Calculate Unconstrained Modal Width

Use previously calculated modal widths for each modal type to generate a total unconstrained modal width value.


**Assumptions, Notes, and Parameters**
1. Future Tier 4 Bus Priority lanes are ignored in the unconstrained width calcuation based on the assumption that Bus Priority lanes will always be placed in existing auto lanes.
2. Do not sum Ped Width and Bike Width if future Bike Type = Shared Use Path
3. Ignore all bike facility types of shared road way in calculation



In [ ]:
#calcualte unconstrained modal widths

fields = ["SEGMENTID",               #0
          "modal_width_ped_prop",    #1
          "modal_width_bike_prop",   #2
          "modal_width_auto_prop",   #3
          "modal_width_bus_prop",    #4
          "modal_width_park_prop",   #5
          "modal_width_medians",     #6
          "modal_width_max",         #7
          "Fac_Type_BP",             #8
          "Fac_Type_BR",             #9
          "NewTier"]                  #10        

count = 0
err_count = 0

#desc

print("Calculating width..")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #igonore shared road-way widths from bike width
            if row[8] == "Shared Roadway" or row[9] == "Shared Roadway":     
                row[7] = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                count +=1
                
            #ignore bike shared use path for ped > 7    
            elif (row[8] == "Shared Use Path" or row[9] == "Shared Use Path") and row[1] is not None and row[1] > 7:
                row[7] = sum(filter(None,(row[1], row[3], row[4], row[5], row[6])))
                count +=1
                
            #if shared use path is proposed on segment where future ped width is less than 7, assume 7 for ped width in calc   
            elif (row[8] == "Shared Use Path" or row[9] == "Shared Use Path") and row[1] is not None and row[1] < 7:
                row[7] = sum(filter(None,(row[1], 7,  row[3], row[4], row[5], row[6])))
                count +=1
            
            #summarize all other segments
            else:
                row[7] = sum(filter(None,(row[1], row[2], row[3], row[4], row[5], row[6])))
                count +=1
                
            #row[7] = sum(filter(None,(row[1], row[2], row[3], row[4], row[5], row[6])))
            #row[7] = sum(filter(None,(row[1], row[2], row[3], row[5], row[6])))
            
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Unconstrained widths calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Unconstrained complete.")


In [ ]:
#add additonal fields for modal width/area metrics

#length_ft - to capture line length in ft prior to conversion to WebMercator. Calculated from shape_length
#mean_row_area - estimated area, 2020 row estimate X length
#modal_area_max - estimated unconstrained area, unconstrained width x length
#modal_area_diff - difference between Unconstrained Area and 2020 Estimate. Uncon - 2020. Positive values will represent exceedences, negative will be surplus 


print("adding fields")
arcpy.AddField_management(modal_composite_05a,field_name="length_ft",field_alias = "Length (ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="mean_row_area",field_alias = "ROW Area Estimate (sq. ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="modal_area_max",field_alias = "Unconstrained Modal Area (sq. ft.)",field_type="FLOAT")
arcpy.AddField_management(modal_composite_05a,field_name="modal_area_diff",field_alias = "Unconstrained Modal Area Exceedence (sq. ft.)",field_type="FLOAT")
print("done")

In [ ]:
list_fields = arcpy.ListFields(modal_composite_05a)
field_names = [i.name for i in list_fields]

field_check = "modal_area_diff"

if field_check in field_names:
    print("field already exists")
else:
    print("not found")
    

In [ ]:
#calculate modal area fields

fields = ["SEGMENTID",       #0
          "modal_width_max", #1
          "Shape_Length",    #2
          "length_ft",       #3
          "mean_row_area",   #4
          "modal_area_max",  #5
          "modal_area_diff", #6
          "mean_row_2020"]   #7
   

count = 0
err_count = 0

#desc

print("Calculating areas..")

with arcpy.da.UpdateCursor(modal_composite, fields, 'NOT "mean_row_2020" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            length_ft = row[2]
            mean_row_area = length_ft * row[7]
            modal_area_max = length_ft * row[1]
            modal_area_diff = modal_area_max - mean_row_area
            
            row[3] = length_ft
            row[4] = mean_row_area
            row[5] = modal_area_max
            row[6] = modal_area_diff
            
            count +=1
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))

print("Modal Areas - complete.")

In [ ]:
#reset all (or some) width and area fields to null (use during development)

fields = ["modal_width_ped_exist",
          "modal_width_ped_prop",
          "modal_width_bike_exist",
          "modal_width_bike_prop",
          "modal_width_auto_exist",
          "modal_width_auto_prop",
          "modal_width_bus_exist",
          "modal_width_bus_prop",
          "modal_width_park_exist",
          "modal_width_park_prop",
          "modal_width_medians",
          "modal_width_max",
          "mean_row_area",
          "modal_area_max",
          "modal_area_diff"]

err_count = 0

print("Reseting modal widths and areas to Null...")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            
#             row[0] = None    # modal_width_ped_exist
#             row[1] = None    # modal_width_ped_prop
#             row[2] = None    # modal_width_bike_exist
#             row[3] = None    # modal_width_bike_prop
#             row[4] = None    # modal_width_auto_exist
#             row[5] = None    # modal_width_auto_prop
#             row[6] = None    # modal_width_bus_exist
#             row[7] = None    # modal_width_bus_prop
#             row[8] = None    # modal_width_park_exist
#             row[9] = None    # modal_width_park_prop
#             row[10] = None   # modal_width_medians
            row[11] = None   # modal_width_max
            row[12] = None   # modal_width_park_prop
            row[13] = None   # modal_width_medians
            row[14] = None   # modal_width_max




            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print(error)

print("All widths and areas reset to Null.")
            

In [ ]:

##OLD
#fields to expose to update cursor for bike facilities
fields = ["Fac_Type_BE","modal_width_bike_exist","Fac_Type_BP","Fac_Type_BR","modal_width_bike_prop"]

#update bike width column
print("Update rows for Bike widths..")
with arcpy.da.UpdateCursor(modal_composite_05a,fields,'NOT ("Fac_Type_BE" IS NULL OR "Fac_Type_BP" IS NULL OR "Fac_Type_BR" IS NULL)') as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike existing calculations
            if row[0] == "Protected Bike Lane" or row[0] == "Buffered Bike Lane":
                row[1] = 10
                
            elif row[0] == "Bike Lane" or row[0] == "Climbing Lane" or row[0] == "Shoulder Bikeway":
                row[1] = 7
            
            #bike proposed & redev calculations
            elif (row[2] == "Protected Bike Lane" or
                  row[2] == "Buffered Bike Lane" or
                  row[3] == "Protected Bike Lane" or
                  row[3] == "Buffered Bike Lane"):
                row[4] = 10
                
            elif (row[2] == "Bike Lane" or
                  row[2] == "Climbing Lane" or
                  row[2] == "Shoulder Bikeway" or
                  row[3] == "Bike Lane" or
                  row[3] == "Climbing Lane" or
                  row[3] == "Shoulder Bikeway"):
                row[4] = 7
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("bike data width calculated\n")
            
#update ln_prop_num w/ update cursor
print("parse ln_prop to int")
with arcpy.da.UpdateCursor(modal_composite_04a,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[1] == '5+ bus':
                row[5] = 5
            else:
                row[5]= int(row[1])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("ln_prop parsed\n")
            
#update row_exist_num w/ update cursor
print("parse row_exist to num")
with arcpy.da.UpdateCursor(modal_composite_04a,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[2] == 'Var. to 80':
                row[6] = 80
            elif row[2] == 'Var. 60-70':
                row[6] = 65
            elif row[2] == 'Var. 36-50':
                row[6] == 61
            elif row[2] == 'Var. 14-26':
                row[6] = 20
            elif row[2] == '86/100':
                row[6] = 93
            elif row[2] == '60-90':
                row[6] = 75
            elif row[2] == '60-76':
                row[6] = 68
            elif row[2] == '60-70':
                row[6] = 65
            elif row[2] == '60-64':
                row[6] = 62
            elif row[2] == '50-80':
                row[6] = 65
            elif row[2] == '50-60':
                row[6] = 55
            elif row[2] == '44-56':
                row[6] = 50
            elif row[2] == '40/50':
                row[6] = 45
            elif row[2] == '40-56':
                row[6] = 48
            elif row[2] == '30-50':
                row[6] = 40
            elif row[2] == '30-40':
                row[6] = 35
            elif row[2] == '25-50':
                row[6] = 38
            elif row[2] == '20-40':
                row[6] = 30
            elif row[2] == '25-50':
                row[6] = 38
            elif row[2] == '120-140':
                row[6] = 130
            elif row[2] == '113.5':
                row[6] = 114
            elif row[2] == '100-110':
                row[6] = 105
            elif row[2] == '0':
                row[6] = None
            else:
                row[6]= int(row[2])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("row_exist parsed\n")            
                  
#update row_prop_num w/ update cursor
print("parse row_prop to int")
with arcpy.da.UpdateCursor(modal_composite_04a,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:          
            if row[3] == 'Var. to 80':
                row[7] = 80
            elif row[3] == 'Var. 14-26':
                row[7] = 20
            elif row[3] == '86/100':
                row[7] = 93
            elif row[3] == '60-90':
                row[7] = 75
            elif row[3] == '60-76':
                row[7] = 68
            elif row[3] == '60-70':
                row[7] = 65
            elif row[3] == '60-64':
                row[7] = 62
            elif row[3] == '56-80':
                row[7] = 68
            elif row[3] == '50-60':
                row[7] = 55
            elif row[3] == '40/50':
                row[7] = 45
            elif row[3] == '30-40':
                row[7] = 35
            elif row[3] == '25-50':
                row[7] = 38
            elif row[3] == '120-140':
                row[7] = 130
            elif row[3] == '113.5':
                row[7] = 114
            elif row[3] == '100-110':
                row[7] = 105
            else:
                row[7] = int(row[3])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("row_prop parsed\n")

print("Processing Complete")